## REMEMBER: Restart this notebook to close connection to `sparkifydb`

Each time you run the cells above, remember to restart this notebook to close the connection to your database. Otherwise, you won't be able to run your code in `create_tables.py`, `etl.py`, or `etl.ipynb` files since you can't make multiple connections to the same database (in this case, sparkifydb).


## Sanity Checks


In [1]:
import os
import sys
import glob
import psycopg2
import pandas as pd
import logging
import warnings

from rich import traceback
from pathlib import Path
from IPython.core import display as ICD


In [2]:
src_path: str = "../src"
sys.path.append(src_path)
_ = traceback.install()
logging.basicConfig(force=True)
logging.getLogger().setLevel(logging.ERROR)
warnings.filterwarnings("ignore")


In [3]:
from create_tables import main as run_create_tables
from etl import main as run_etl
from sql_queries import get_simple_select_query, TABLES


### Table creation


Table creation script runs without errors.


In [4]:
run_create_tables()


Fact and dimensional tables for a star schema are properly defined.


In [5]:
conn = psycopg2.connect(
    "host=127.0.0.1 dbname=sparkifydb user=student password=student"
)
cur = conn.cursor()
query_columns = ("table_name", "column_name", "data_type")


for table_name in TABLES.keys():
    cur.execute(
        get_simple_select_query(
            query_columns,
            "information_schema.columns",
            {"table_name": f"'{table_name}'"},
        )
    )
    conn.commit()
    print(f"Description of table {table_name}:")
    ICD.display(
        pd.DataFrame(cur.fetchall(), columns=query_columns)
        .set_index("column_name")["data_type"]
        .to_frame()
    )
    print("\n\n")

conn.commit()
conn.close()


Description of table songplays:


,data_type
column_name,
start_time,timestamp without time zone
session_id,integer
user_id,integer
songplay_id,integer
location,text
user_agent,text
level,text
song_id,text
artist_id,text





Description of table users:


,data_type
column_name,
user_id,integer
first_name,text
last_name,text
gender,text
level,text





Description of table songs:


,data_type
column_name,
year,integer
duration,numeric
song_id,text
title,text
artist_id,text





Description of table artists:


,data_type
column_name,
latitude,numeric
longitude,numeric
artist_id,text
name,text
location,text





Description of table time:


,data_type
column_name,
start_time,timestamp without time zone
hour,integer
day,integer
week,integer
month,integer
year,integer
weekday,integer


### ETL


ETL script runs without errors.


In [6]:
run_etl()


Output()

Output()

ETL script properly processes transformations in Python.


In [7]:
conn = psycopg2.connect(
    "host=127.0.0.1 dbname=sparkifydb user=student password=student"
)
cur = conn.cursor()
query_columns = ("table_name", "column_name", "data_type")


for table_name in TABLES.keys():
    cur.execute(
        get_simple_select_query(
            query_columns,
            "information_schema.columns",
            {"table_name": f"'{table_name}'"},
        )
    )
    conn.commit()

    table_columns = pd.DataFrame(cur.fetchall(), columns=query_columns)["column_name"]

    cur.execute(get_simple_select_query(("*",), table_name, limit=5))
    conn.commit()
    print(f"Description of table {table_name}:")
    ICD.display(pd.DataFrame(cur.fetchall(), columns=table_columns))
    print("\n\n")

cur.close()
conn.close()


Description of table songplays:


column_name,start_time,session_id,user_id,songplay_id,location,user_agent,level,song_id,artist_id
0,1,2018-11-13 00:40:37.796,66,free,None,None,514,"Harrisburg-Carlisle, PA","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4..."
1,3,2018-11-13 01:12:29.796,51,free,None,None,510,"Houston-The Woodlands-Sugar Land, TX","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
2,4,2018-11-13 03:19:02.796,9,free,None,None,379,"Eureka-Arcata-Fortuna, CA",Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7....
3,5,2018-11-13 03:51:52.796,49,free,None,None,506,"San Francisco-Oakland-Hayward, CA",Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...
4,8,2018-11-13 05:00:06.796,94,free,None,None,492,"Ogden-Clearfield, UT",Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...





Description of table users:


column_name,user_id,first_name,last_name,gender,level
0,66,Kevin,Arellano,M,free
1,51,Maia,Burke,F,free
2,9,Wyatt,Scott,M,free
3,49,Chloe,Cuevas,F,free
4,94,Noah,Chavez,M,free





Description of table songs:


column_name,year,duration,song_id,title,artist_id
0,SONSKXP12A8C13A2C9,Native Soul,AR0IAWL1187B9A96D0,2003,197.19791
1,SORRZGD12A6310DBC3,Harajuku Girls,ARVBRGZ1187FB4675A,2004,290.55955
2,SOWQTQZ12A58A7B63E,Streets On Fire (Explicit Album Version),ARPFHN61187FB575F6,0,279.97995
3,SODREIN12A58A7F2E5,A Whiter Shade Of Pale (Live @ Fillmore West),ARLTWXK1187FB5A3F8,0,326.00771
4,SOFCHDR12AB01866EF,Living Hell,AREVWGE1187B9B890A,0,282.43546





Description of table artists:


column_name,latitude,longitude,artist_id,name,location
0,AR0IAWL1187B9A96D0,Danilo Perez,Panama,8.4177,-80.11278
1,ARVBRGZ1187FB4675A,Gwen Stefani,,None,None
2,ARPFHN61187FB575F6,Lupe Fiasco,"Chicago, IL",41.88415,-87.63241
3,ARLTWXK1187FB5A3F8,King Curtis,"Fort Worth, TX",32.74863,-97.32925
4,AREVWGE1187B9B890A,Bitter End,Noci (BA),-13.442,-41.9952





Description of table time:


column_name,start_time,hour,day,week,month,year,weekday
0,2018-11-13 00:40:37.796,0,13,46,11,2018,1
1,2018-11-13 01:12:29.796,1,13,46,11,2018,1
2,2018-11-13 03:19:02.796,3,13,46,11,2018,1
3,2018-11-13 03:51:52.796,3,13,46,11,2018,1
4,2018-11-13 05:00:06.796,5,13,46,11,2018,1
